In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.transforms import ToUndirected, RandomLinkSplit
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler
from torch.optim import Adam
from sklearn.metrics.pairwise import cosine_similarity
import time
from sklearn.metrics import roc_auc_score

In [8]:
df = pd.read_csv('./news_final.csv', encoding='cp949')

FileNotFoundError: [Errno 2] No such file or directory: './news_final.csv'

In [3]:
# 임시 종목 파싱 코드
import lxml
import urllib.request  # urllib.request 모듈 추가
import re

# KRX 상장법인 현황 페이지 URL
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download'

try:
    response = urllib.request.urlopen(url)
    data_bytes = response.read()

    df_krx = pd.read_html(
        data_bytes,  
        header=0,
        converters={'종목코드': str},
        flavor='lxml',
        encoding='cp949' 
    )[0]

    # '회사명' 컬럼을 리스트로 변환
    stock_name_list = df_krx['회사명'].tolist()

except ImportError:
    print("패키지가 설치되지 않았습니다. 'uv pip install pandas lxml'을 실행해주세요.")
except Exception as e:
    print(f"데이터를 가져오는 중 오류가 발생했습니다: {e}")
    
# 1. 정규식 패턴 생성
#    - re.escape(): '(주)'나 'LG+U'처럼 정규식 특수문자가 포함된 회사명을 안전하게 처리
#    - .sort(key=len, reverse=True): 'SK하이닉스'가 'SK'보다 먼저 매칭되도록 긴 이름순으로 정렬
escaped_names = sorted([re.escape(name) for name in stock_name_list], key=len, reverse=True)
regex_pattern = '|'.join(escaped_names)

print(f"생성된 정규식 패턴 (일부): {regex_pattern[:100]}...")

# 2. findall로 text 컬럼에서 모든 회사명을 리스트로 추출
#    - .str.findall()은 각 행에서 매칭되는 모든 이름을 리스트로 반환
matches = df['text'].str.findall(regex_pattern)

# 3. 추출된 리스트에서 중복 제거 (순서 유지)
#    - .apply()를 사용해 각 행의 리스트(x)에 대해 dict.fromkeys를 적용
#    - dict.fromkeys는 순서를 유지하면서 중복을 제거하는 가장 빠른 방법입니다.
#    - isinstance(x, list)는 원본 text가 NaN일 경우 오류가 나는 것을 방지합니다.
unique_matches = matches.apply(
    lambda x: list(dict.fromkeys(x)) if isinstance(x, list) else []
)

# 4. 리스트를 ','로 구분된 문자열로 변환하여 새 컬럼에 할당
#    - .str.join(',')은 ['삼성전자', 'SK하이닉스'] -> '삼성전자,SK하이닉스'로 바꿔줌
df['stock'] = unique_matches.str.join(',')

#5. keyword 열 삭제(임시)
df = df.drop('keyword', axis=1)
print("\n--- 결과 ---")
print(df)

생성된 정규식 패턴 (일부): SM\ Life\ Design|피엔케이피부임상연구센타|대신밸런스제19호스팩|대신밸런스제18호스팩|대신밸런스제17호스팩|대신밸런스제16호스팩|큐리옥스바이오시스템즈|프레스티지바이오로직...

--- 결과 ---
      crawled_news_id                                          title  \
0               16388            계엄날 한동훈 체포조장 , 포고문 버리고 편의점에서 시간 끌었다   
1               16391                         "장난치면 패가망신" 주가조작 칼 빼든다   
2               16387  "깨 많이, 콩 많이" 경쟁하며 송편 빚고, 전 부치던 부엌 응답하라 90'...   
3               16397             강훈식 "김현지, 직언 참모 인사, 이미 한 달 전부터 준비"   
4               16386         5일째 파업 인천공항 노동자들 공항 모·자회사, 시민·직원 불편...   
...               ...                                            ...   
7538            24296         조주완 LG전자 CEO, 사우디 정부와 네옴시티 냉각솔루션 협력 논의   
7539            24295           민형배 의원, '글로컬대학 결과' SNS 올렸다가 삭제 '구설수'   
7540            24301              보험직썰 미국계 달튼 vs 토종파 MBK 이들 출자자 차이는   
7541            24294                농협재단, 우리 쌀 나눔으로 취약계층과 농업인 행복 지원   
7542            24293                      농협 축산경제, 농·축협 수의사 기술교육 실시   

  

In [9]:
## 1단계: 엣지(Edge) 전처리
# 목표: 쉼표로 묶인 문자열을 분해하여 (뉴스 ID, 키워드) / (뉴스 ID, 종목) 엣지 리스트 생성

def split_and_clean(series):
    """쉼표로 구분된 문자열을 분리하고, 공백을 제거하며, 빈 문자열을 필터링합니다."""
    return series.str.split(',').apply(
        lambda x: [item.strip() for item in x if item.strip()]
    )

# 1. 키워드 엣지 생성
df['keywords_list'] = split_and_clean(df['tfidf_keywords'])
# (수정) 이름 충돌 방지를 위해 필요한 컬럼만 먼저 선택
news_to_keyword_edges = df[['crawled_news_id', 'keywords_list']] \
    .explode('keywords_list') \
    .rename(columns={'keywords_list': 'keyword'}) \
    .dropna()

# 2. 종목 엣지 생성
df['stock_list'] = split_and_clean(df['stock'])
# (수정) 이름 충돌 방지를 위해 필요한 컬럼만 먼저 선택
news_to_stock_edges = df[['crawled_news_id', 'stock_list']] \
    .explode('stock_list') \
    .rename(columns={'stock_list': 'stock'}) \
    .dropna()

print("--- [1. 엣지 전처리 완료] ---")
print(f"뉴스-키워드 엣지 수: {len(news_to_keyword_edges)}")
print(f"뉴스-종목 엣지 수: {len(news_to_stock_edges)}\n")

--- [1. 엣지 전처리 완료] ---
뉴스-키워드 엣지 수: 75379
뉴스-종목 엣지 수: 10953


In [12]:
## 2단계: 노드(Node) 및 특징(Feature) 생성
# 목표: 3가지 노드(news, keyword, stock)를 정의하고, 숫자 벡터(특징)로 변환

# 1. 고유 노드 및 매핑 생성
# (중요) GNN은 0부터 시작하는 정수 인덱스를 사용합니다.
# 원본 ID(crawled_news_id, '삼성전자') -> GNN용 ID(0, 1, 2...)로 변환

# 뉴스 노드
# 'crawled_news_id'를 GNN 인덱스(0 ~ len(df)-1)에 매핑
news_id_map = {news_id: i for i, news_id in enumerate(df['crawled_news_id'])}

# 키워드 노드
unique_keywords = news_to_keyword_edges['keyword'].unique()
keyword_map = {keyword: i for i, keyword in enumerate(unique_keywords)}

# 종목 노드
unique_stocks = news_to_stock_edges['stock'].unique()
stock_map = {stock: i for i, stock in enumerate(unique_stocks)}

print("--- [2. 노드 정의 완료] ---")
print(f"뉴스 노드 수: {len(news_id_map)}")
print(f"키워드 노드 수: {len(keyword_map)}")
print(f"종목 노드 수: {len(stock_map)}\n")

# 2. 노드 특징(Feature) 생성
print("--- [2. 특징 생성 시작] ---")

# (A) 'news' 노드 특징: (Title Embedding + Scaled PubDate)
# Title 임베딩 (SBERT)
print("  (A-1) 'news' 노드: Title 임베딩 중... (시간 소요)")
sbert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# 'df'의 순서가 'news_id_map'의 0~N 순서와 동일하다고 가정하고 진행
title_embeddings = sbert_model.encode(df['title'].tolist(), show_progress_bar=True)

# PubDate (최신성)
print("  (A-2) 'news' 노드: PubDate 스케일링 중...")
# 날짜를 숫자로 변환 (Unix timestamp)
pub_date_numeric = pd.to_datetime(df['pub_date']).astype('int64').values.reshape(-1, 1)
# StandardScaler로 정규화 (평균 0, 분산 1)
scaler = StandardScaler()
pub_date_scaled = scaler.fit_transform(pub_date_numeric)

# Title 임베딩과 날짜 특징 결합
news_features = np.hstack([title_embeddings, pub_date_scaled])
news_feat_tensor = torch.tensor(news_features, dtype=torch.float)
news_feat_dim = news_features.shape[1] # GNN 모델에 전달할 차원 수

# (B) 'keyword' 노드 특징: 고유 ID
# 'keyword' 자체는 특징이 없으므로, GNN이 스스로 학습할 수 있도록
# Embedding 레이어를 사용할 것입니다.
# 여기서는 0부터 N-1까지의 인덱스를 특징으로 전달합니다.
keyword_feat_tensor = torch.arange(len(unique_keywords))

# (C) 'stock' 노드 특징: 고유 ID
# 'keyword'와 동일하게 Embedding 레이어 사용
stock_feat_tensor = torch.arange(len(unique_stocks))

print("--- [2. 특징 생성 완료] ---")
print(f"뉴스 특징 벡터 차원: {news_feat_tensor.shape}")
print(f"키워드 특징 벡터 차원: {keyword_feat_tensor.shape}")
print(f"종목 특징 벡터 차원: {stock_feat_tensor.shape}\n")

--- [2. 노드 정의 완료] ---
뉴스 노드 수: 7543
키워드 노드 수: 4793
종목 노드 수: 562

--- [2. 특징 생성 시작] ---
  (A-1) 'news' 노드: Title 임베딩 중... (시간 소요)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\dobi\Desktop\study\DataPipeline\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dobi\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/236 [00:00<?, ?it/s]

  (A-2) 'news' 노드: PubDate 스케일링 중...
--- [2. 특징 생성 완료] ---
뉴스 특징 벡터 차원: torch.Size([7543, 385])
키워드 특징 벡터 차원: torch.Size([4793])
종목 특징 벡터 차원: torch.Size([562])


In [13]:
## 3단계: 그래프(Graph) 생성
# 목표: 1, 2단계의 결과를 PyTorch Geometric의 'HeteroData' 객체에 통합

data = HeteroData()

# 1. 노드 특징 할당
data['news'].x = news_feat_tensor
data['keyword'].x = keyword_feat_tensor
data['stock'].x = stock_feat_tensor

# 2. 엣지 인덱스 생성 및 할당
# (뉴스 -> 키워드)
src_news_kw = news_to_keyword_edges['crawled_news_id'].map(news_id_map).values
dst_keyword = news_to_keyword_edges['keyword'].map(keyword_map).values
data['news', 'links_to', 'keyword'].edge_index = torch.tensor(
    [src_news_kw, dst_keyword], dtype=torch.long
)

# (뉴스 -> 종목)
src_news_st = news_to_stock_edges['crawled_news_id'].map(news_id_map).values
dst_stock = news_to_stock_edges['stock'].map(stock_map).values
data['news', 'mentions', 'stock'].edge_index = torch.tensor(
    [src_news_st, dst_stock], dtype=torch.long
)

# 3. 양방향 그래프로 변환
# GNN은 메시지 패싱을 위해 양방향 엣지가 있을 때 학습이 잘 됩니다.
# (news -> keyword) 뿐만 아니라 (keyword -> news) 엣지도 추가합니다.
data = ToUndirected()(data)

print("--- [3. 그래프 생성 완료] ---")
print(f"HeteroData 객체:\n{data}\n")
print(f"그래프 엣지 타입: {data.edge_types}\n")


--- [3. 그래프 생성 완료] ---
HeteroData 객체:
HeteroData(
  news={ x=[7543, 385] },
  keyword={ x=[4793] },
  stock={ x=[562] },
  (news, links_to, keyword)={ edge_index=[2, 75379] },
  (news, mentions, stock)={ edge_index=[2, 10953] },
  (keyword, rev_links_to, news)={ edge_index=[2, 75379] },
  (stock, rev_mentions, news)={ edge_index=[2, 10953] }
)

그래프 엣지 타입: [('news', 'links_to', 'keyword'), ('news', 'mentions', 'stock'), ('keyword', 'rev_links_to', 'news'), ('stock', 'rev_mentions', 'news')]


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16400\3366499819.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  data['news', 'links_to', 'keyword'].edge_index = torch.tensor(


In [18]:
## 4단계: GNN 모델 설계 및 훈련 (수정)
# 목표: '링크 예측' 태스크로 GNN을 훈련시켜 '연관성'을 학습

# (수정) to_hetero 대신 HeteroConv를 임포트합니다.
from torch_geometric.nn import SAGEConv, to_hetero, HeteroConv
from torch.optim import Adam
from sklearn.metrics import roc_auc_score

# --- GNN 모델 정의 ---
# 1. 인코더 (Encoder): 노드를 임베딩 벡터로 요약
class GNNEncoder(torch.nn.Module):
    # (수정) __init__에서 metadata가 더 이상 필요하지 않습니다.
    def __init__(self, hidden_dim, out_dim, news_feat_dim, num_keywords, num_stocks):
        super().__init__()
        
        # 'keyword'와 'stock'은 특징이 없으므로(ID뿐), 학습 가능한 Embedding 레이어 생성
        self.keyword_emb = torch.nn.Embedding(num_keywords, hidden_dim)
        self.stock_emb = torch.nn.Embedding(num_stocks, hidden_dim)
        
        # 'news'는 특징이 있으므로(SBERT+Date), Linear 레이어로 차원 통일
        self.news_lin = torch.nn.Linear(news_feat_dim, hidden_dim)
        
        # (수정) to_hetero 대신 HeteroConv를 사용합니다.
        # HeteroConv는 각 엣지 타입별로 사용할 GNN 레이어를 명시적으로 정의합니다.
        
        # Layer 1: 입력 -> hidden_dim
        self.conv1 = HeteroConv({
            ('news', 'links_to', 'keyword'): SAGEConv(-1, hidden_dim),
            ('keyword', 'rev_links_to', 'news'): SAGEConv(-1, hidden_dim),
            ('news', 'mentions', 'stock'): SAGEConv(-1, hidden_dim),
            ('stock', 'rev_mentions', 'news'): SAGEConv(-1, hidden_dim),
        }, aggr='sum')

        # Layer 2: hidden_dim -> out_dim
        self.conv2 = HeteroConv({
            ('news', 'links_to', 'keyword'): SAGEConv(-1, out_dim),
            ('keyword', 'rev_links_to', 'news'): SAGEConv(-1, out_dim),
            ('news', 'mentions', 'stock'): SAGEConv(-1, out_dim),
            ('stock', 'rev_mentions', 'news'): SAGEConv(-1, out_dim),
        }, aggr='sum')


    def forward(self, x_dict, edge_index_dict):
        # 1. 초기 특징 변환 (Embedding / Linear)
        x_dict = {
            'news': self.news_lin(x_dict['news']).relu(), # ReLU 추가
            'keyword': self.keyword_emb(x_dict['keyword']),
            'stock': self.stock_emb(x_dict['stock'])
        }
        
        # 2. GNN 레이어 (HeteroConv는 to_hetero 없이 바로 사용)
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: x.relu() for key, x in x_dict.items()}
        
        x_dict = self.conv2(x_dict, edge_index_dict)
        
        return x_dict # 최종 임베딩 딕셔너리

# 2. 디코더 (Decoder): 두 노드 임베딩으로 '링크' 여부 예측
class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        # 두 임베딩을 받아 1개의 값(링크 확률)으로 변환
        self.lin1 = torch.nn.Linear(2 * hidden_dim, hidden_dim)
        self.lin2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, z_src, z_dst):
        z = torch.cat([z_src, z_dst], dim=-1)
        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

# 3. 전체 모델 (Encoder + Decoder)
class Model(torch.nn.Module):
    # (수정) __init__에서 metadata가 더 이상 필요하지 않습니다.
    def __init__(self, hidden_dim, out_dim, news_feat_dim, num_keywords, num_stocks):
        super().__init__()
        self.encoder = GNNEncoder(
            hidden_dim, out_dim, news_feat_dim, num_keywords, num_stocks
        )
        # 2가지 엣지(news-keyword, news-stock)를 예측할 디코더
        self.decoders = torch.nn.ModuleDict({
            'links_to': EdgeDecoder(out_dim),
            'mentions': EdgeDecoder(out_dim),
        })

    def forward(self, x_dict, edge_index_dict, edge_label_index_dict):
        z_dict = self.encoder(x_dict, edge_index_dict) # 최종 임베딩
        
        preds = {}
        for edge_type, edge_label_index in edge_label_index_dict.items():
            src_type, rel_type, dst_type = edge_type
            
            z_src = z_dict[src_type][edge_label_index[0]]
            z_dst = z_dict[dst_type][edge_label_index[1]]
            
            preds[edge_type] = self.decoders[rel_type](z_src, z_dst)
            
        return preds

# --- 훈련 준비 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"--- [4. GNN 훈련 시작] ---\n사용 디바이스: {device}\n")

# 1. 훈련/검증/테스트 데이터 분할 (링크 예측용)
# (주의) data 객체를 변환하므로, 이 분할은 'data' 객체에 대해 한 번만 실행해야 합니다.
# 만약 코드를 여러 번 실행했다면, '3단계: 그래프 생성' 부터 다시 실행하는 것이 안전합니다.
transform = RandomLinkSplit(
    num_val=0.1,  # 10% 검증
    num_test=0.1, # 10% 테스트
    is_undirected=True,
    add_negative_train_samples=True, # (중요) '없는 엣지'도 샘플링
    edge_types=[('news', 'links_to', 'keyword'), ('news', 'mentions', 'stock')],
    rev_edge_types=[('keyword', 'rev_links_to', 'news'), ('stock', 'rev_mentions', 'news')],
)
train_data, val_data, test_data = transform(data)

# 2. 모델, 옵티마이저, 손실함수 초기화
HIDDEN_DIM = 64
OUT_DIM = 32 # 최종 임베딩 차원

model = Model(
    hidden_dim=HIDDEN_DIM,
    out_dim=OUT_DIM,
    news_feat_dim=news_feat_dim,
    num_keywords=len(unique_keywords),
    num_stocks=len(unique_stocks)
    # (수정) metadata 인자 제거
).to(device)

optimizer = Adam(model.parameters(), lr=0.01)
# 긍정/부정 링크(엣지)를 분류하는 이진 분류 문제
criterion = torch.nn.BCEWithLogitsLoss()

# --- 훈련 함수 ---
def train(data):
    model.train()
    optimizer.zero_grad()
    
    # 훈련용 엣지(pos/neg)
    edge_label_index_dict = {
        ('news', 'links_to', 'keyword'): data['news', 'links_to', 'keyword'].edge_label_index,
        ('news', 'mentions', 'stock'): data['news', 'mentions', 'stock'].edge_label_index
    }
    
    # 모델 예측
    preds_dict = model(data.x_dict, data.edge_index_dict, edge_label_index_dict)
    
    # 정답 레이블
    labels_dict = {
        ('news', 'links_to', 'keyword'): data['news', 'links_to', 'keyword'].edge_label,
        ('news', 'mentions', 'stock'): data['news', 'mentions', 'stock'].edge_label
    }

    # 2가지 엣지 타입의 손실을 합산
    loss = 0
    for edge_type in preds_dict:
        loss += criterion(preds_dict[edge_type], labels_dict[edge_type].float())
        
    loss.backward()
    optimizer.step()
    return float(loss)

# --- 검증 함수 ---
@torch.no_grad()
def evaluate(data):
    model.eval()
    
    edge_label_index_dict = {
        ('news', 'links_to', 'keyword'): data['news', 'links_to', 'keyword'].edge_label_index,
        ('news', 'mentions', 'stock'): data['news', 'mentions', 'stock'].edge_label_index
    }
    preds_dict = model(data.x_dict, data.edge_index_dict, edge_label_index_dict)
    
    labels_dict = {
        ('news', 'links_to', 'keyword'): data['news', 'links_to', 'keyword'].edge_label,
        ('news', 'mentions', 'stock'): data['news', 'mentions', 'stock'].edge_label
    }
    
    total_auc = 0
    for edge_type in preds_dict:
        preds = preds_dict[edge_type].sigmoid().cpu().numpy()
        labels = labels_dict[edge_type].cpu().numpy()
        total_auc += roc_auc_score(labels, preds)
        
    return total_auc / len(preds_dict) # 평균 AUC

# --- 훈련 루프 실행 ---
train_data, val_data = train_data.to(device), val_data.to(device)
best_val_auc = 0

for epoch in range(1, 101): # 100 에포크
    loss = train(train_data)
    val_auc = evaluate(val_data)
    
    if epoch % 10 == 0:
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}")
    
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        # (선택) 최고 성능 모델 저장
        # torch.save(model.state_dict(), 'best_model.pth')

print(f"\n훈련 완료! 최고 검증 AUC: {best_val_auc:.4f}\n")

--- [4. GNN 훈련 시작] ---
사용 디바이스: cuda


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16400\2162690416.py:162: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:836.)
  return float(loss)


Epoch: 010, Loss: 0.9648, Val AUC: 0.7693
Epoch: 020, Loss: 0.7590, Val AUC: 0.8215
Epoch: 030, Loss: 0.6131, Val AUC: 0.8466
Epoch: 040, Loss: 0.5093, Val AUC: 0.8589
Epoch: 050, Loss: 0.4333, Val AUC: 0.8618
Epoch: 060, Loss: 0.3805, Val AUC: 0.8645
Epoch: 070, Loss: 0.3288, Val AUC: 0.8684
Epoch: 080, Loss: 0.2837, Val AUC: 0.8718
Epoch: 090, Loss: 0.2516, Val AUC: 0.8725
Epoch: 100, Loss: 2.5061, Val AUC: 0.8728

훈련 완료! 최고 검증 AUC: 0.8763


In [19]:
## 5단계: 최종 임베딩 추출
# 목표: 훈련된 GNN Encoder를 사용하여 모든 노드의 최종 임베딩 벡터 추출

print("--- [5. 최종 임베딩 추출] ---")

# (중요) 훈련 시 사용한 'train_data'가 아닌,
# 모든 엣지 정보가 포함된 원본 'data' 객체를 사용합니다.
data = data.to(device)

model.eval()
with torch.no_grad():
    # model.encoder만 호출하여 GNN을 통과한 최종 노드 임베딩을 얻습니다.
    final_embeddings = model.encoder(data.x_dict, data.edge_index_dict)

# 결과 확인
print("임베딩 추출 성공!")
for node_type, embedding_tensor in final_embeddings.items():
    print(f"  - '{node_type}' 임베딩 크기: {list(embedding_tensor.shape)}")

# CPU로 이동
final_embeddings_cpu = {
    key: tensor.cpu().numpy() for key, tensor in final_embeddings.items()
}
# 'final_embeddings' 딕셔너리가 최종 산출물입니다.
# 
# final_embeddings_cpu['news'] -> (7543, 32) 크기의 Numpy 배열
# final_embeddings_cpu['keyword'] -> (len(unique_keywords), 32) 크기의 Numpy 배열
# final_embeddings_cpu['stock'] -> (len(unique_stocks), 32) 크기의 Numpy 배열
#
# 이 벡터들을 Vector DB에 저장하여 유사도 검색에 활용할 수 있습니다.
# (예: 'AI' 키워드 벡터와 가장 가까운 'stock' 벡터 찾기)

print("\n--- [작업 완료] ---")
print("이제 'final_embeddings_cpu' 딕셔너리의 벡터들을 Vector DB에 저장할 수 있습니다.")

--- [5. 최종 임베딩 추출] ---
임베딩 추출 성공!
  - 'keyword' 임베딩 크기: [4793, 32]
  - 'news' 임베딩 크기: [7543, 32]
  - 'stock' 임베딩 크기: [562, 32]

--- [작업 완료] ---
이제 'final_embeddings_cpu' 딕셔너리의 벡터들을 Vector DB에 저장할 수 있습니다.


In [24]:

# --- [0. 사전 준비: 전체 탐색 대상(Corpus) 구축] ---
# (이전 단계의 final_embeddings_cpu, keyword_map, stock_map, unique_keywords 변수가 
#  메모리에 로드되어 있다고 가정합니다.)

print("--- [0. 전체 탐색 대상 구축 시작] ---")
start_time = time.time()

all_nodes_list = []    # (이름, 타입)
all_vectors_list = []  # 벡터
query_keywords = []    # 1번 작업용 쿼리 리스트
query_stocks = []      # 2번 작업용 쿼리 리스트

# 1. 모든 '키워드' 노드 추가
keyword_matrix = final_embeddings_cpu['keyword']
# unique_keywords는 2단계에서 생성된 이름 리스트입니다.
for i, name in enumerate(unique_keywords):
    if name and name.strip(): # (혹시 모를 빈 키워드 제외)
        all_nodes_list.append((name, '키워드'))
        all_vectors_list.append(keyword_matrix[i])
        query_keywords.append(name) # 쿼리 리스트에도 추가

# 2. 모든 '유효한 종목' 노드 추가
stock_matrix = final_embeddings_cpu['stock']
# stock_map은 2단계에서 생성된 {이름: 인덱스} 딕셔너리입니다.
for name, idx in stock_map.items():
    if name and name.strip(): # ''(빈 문자열)이 아닌 유효한 종목만
        all_nodes_list.append((name, '종목'))
        all_vectors_list.append(stock_matrix[idx])
        query_stocks.append(name) # 쿼리 리스트에도 추가

# 3. Numpy 행렬로 최종 변환
all_vectors_matrix = np.vstack(all_vectors_list)
total_nodes = len(all_nodes_list)

print(f"총 {total_nodes}개 노드 (키워드 {len(query_keywords)} + 유효 종목 {len(query_stocks)})의 임베딩 행렬 구축 완료.")
print(f"소요 시간: {time.time() - start_time:.2f}초")


# --- [헬퍼 함수: Top-K 찾기] ---
def find_top_k_similar(query_name, query_type, k=20):
    """
    하나의 쿼리(이름, 타입)를 받아,
    미리 구축된 'all_vectors_matrix'와 비교하여 
    가장 유사한 K개를 반환합니다.
    """
    
    # 1. 쿼리 벡터 찾기
    query_vector = None
    if query_type == '키워드':
        if query_name in keyword_map:
            query_vector = final_embeddings_cpu['keyword'][keyword_map[query_name]]
    elif query_type == '종목':
        if query_name in stock_map:
            query_vector = final_embeddings_cpu['stock'][stock_map[query_name]]
    
    if query_vector is None:
        print(f"  오류: '{query_name}'의 벡터를 찾을 수 없습니다.")
        return []

    # 2. 코사인 유사도 계산
    # (1, N) @ (N, M) -> (1, M)
    # N = embedding_dim, M = total_nodes
    sim_scores = cosine_similarity(query_vector.reshape(1, -1), all_vectors_matrix)[0]
    
    # 3. 상위 K개 인덱스 추출 (내림차순)
    top_indices = np.argsort(sim_scores)[::-1]
    
    # 4. 결과 리스트 생성 (자기 자신 제외)
    results = []
    for idx in top_indices:
        if len(results) >= k:
            break
            
        # all_nodes_list에서 정보 가져오기
        item_name, item_type = all_nodes_list[idx]
        
        # 자기 자신은 건너뛰기
        if item_name == query_name:
            continue
            
        score = sim_scores[idx]
        results.append((item_name, item_type, score))
        
    return results

# --- [1번 작업: 키워드 10개 -> Top 20] ---
print("\n" + "=" * 50)
print("🔍 [작업 1] 랜덤 키워드 10개에 대한 TOP 20 유사 노드")
print("=" * 50)

N_QUERIES = 10

# 1. 쿼리할 키워드 10개 샘플링
if len(query_keywords) < N_QUERIES:
    print(f"경고: 키워드 수가 {N_QUERIES}개 미만이라 전체 키워드를 사용합니다.")
    sampled_keywords = query_keywords
else:
    sampled_keywords = np.random.choice(query_keywords, N_QUERIES, replace=False)

# 2. 10개 쿼리 실행
for i, query_name in enumerate(sampled_keywords):
    print(f"\n--- [1-{i+1:2d}] 쿼리: '{query_name}' (키워드) ---")
    
    start_time = time.time()
    top_k_results = find_top_k_similar(query_name, '키워드', k=20)
    
    if not top_k_results:
        print("  유사한 노드를 찾지 못했습니다.")
        continue
        
    for rank, (name, n_type, score) in enumerate(top_k_results):
        print(f"  {rank+1:2d}위. {name} ({n_type}) (유사도: {score:.4f})")
    print(f"  (소요 시간: {time.time() - start_time:.2f}초)")


# --- [2번 작업: 종목 10개 -> Top 20] ---
print("\n" + "=" * 50)
print("📈 [작업 2] 랜덤 종목 10개에 대한 TOP 20 유사 노드")
print("=" * 50)

# 1. 쿼리할 종목 10개 샘플링
if len(query_stocks) < N_QUERIES:
    print(f"경고: 종목 수가 {N_QUERIES}개 미만이라 전체 유효 종목을 사용합니다.")
    sampled_stocks = query_stocks
else:
    sampled_stocks = np.random.choice(query_stocks, N_QUERIES, replace=False)

# 2. 10개 쿼리 실행
for i, query_name in enumerate(sampled_stocks):
    print(f"\n--- [2-{i+1:2d}] 쿼리: '{query_name}' (종목) ---")
    
    start_time = time.time()
    top_k_results = find_top_k_similar(query_name, '종목', k=20)
    
    if not top_k_results:
        print("  유사한 노드를 찾지 못했습니다.")
        continue
        
    for rank, (name, n_type, score) in enumerate(top_k_results):
        print(f"  {rank+1:2d}위. {name} ({n_type}) (유사도: {score:.4f})")
    print(f"  (소요 시간: {time.time() - start_time:.2f}초)")

print("\n" + "=" * 50)
print("모든 작업 완료.")

--- [0. 전체 탐색 대상 구축 시작] ---
총 5355개 노드 (키워드 4793 + 유효 종목 562)의 임베딩 행렬 구축 완료.
소요 시간: 0.00초

🔍 [작업 1] 랜덤 키워드 10개에 대한 TOP 20 유사 노드

--- [1- 1] 쿼리: '인기' (키워드) ---
   1위. 배우 (키워드) (유사도: 0.9783)
   2위. 드라마 (키워드) (유사도: 0.9782)
   3위. 시청자 (키워드) (유사도: 0.9778)
   4위. 연기 (키워드) (유사도: 0.9763)
   5위. 연애 (키워드) (유사도: 0.9751)
   6위. 영화 (키워드) (유사도: 0.9729)
   7위. 조용필 (키워드) (유사도: 0.9663)
   8위. 숙소 (키워드) (유사도: 0.9638)
   9위. 발매 (키워드) (유사도: 0.9634)
  10위. 기사 (키워드) (유사도: 0.9627)
  11위. 캐스팅 (키워드) (유사도: 0.9624)
  12위. 인간 (키워드) (유사도: 0.9624)
  13위. 부작 (키워드) (유사도: 0.9599)
  14위. 미리 (키워드) (유사도: 0.9590)
  15위. 정말 (키워드) (유사도: 0.9589)
  16위. 관객 (키워드) (유사도: 0.9582)
  17위. 주인공 (키워드) (유사도: 0.9570)
  18위. 재회 (키워드) (유사도: 0.9566)
  19위. 셰프 (키워드) (유사도: 0.9565)
  20위. 지영 (키워드) (유사도: 0.9563)
  (소요 시간: 0.00초)

--- [1- 2] 쿼리: '야외' (키워드) ---
   1위. 규범 (키워드) (유사도: 0.9531)
   2위. 폭우 (키워드) (유사도: 0.9482)
   3위. 표적 (키워드) (유사도: 0.9455)
   4위. 관찰 (키워드) (유사도: 0.9450)
   5위. 미숙 (키워드) (유사도: 0.9450)
   6위. 신부 (키워드) (유사도: 0.9393)
   7위. 스

In [28]:
# --- [헬퍼 함수: Top-K '필터링' 찾기] ---
def find_top_k_filtered(query_name, query_type, target_type, k=20):
    """
    하나의 쿼리(이름, 타입)를 받아,
    전체 노드와 비교 후 'target_type'에 해당하는
    가장 유사한 K개를 반환합니다.
    
    Args:
        query_name (str): 쿼리할 단어 (예: '삼성전자')
        query_type (str): 쿼리 단어의 타입 (예: '종목')
        target_type (str): 찾고자 하는 결과의 타입 (예: '키워드')
        k (int): 상위 K개
    """
    
    # 1. 쿼리 벡터 찾기
    query_vector = None
    if query_type == '키워드':
        if query_name in keyword_map:
            query_vector = final_embeddings_cpu['keyword'][keyword_map[query_name]]
    elif query_type == '종목':
        if query_name in stock_map:
            query_vector = final_embeddings_cpu['stock'][stock_map[query_name]]
    
    if query_vector is None:
        print(f"  오류: '{query_name}'의 벡터를 찾을 수 없습니다.")
        return []

    # 2. 코사인 유사도 계산 (전체 노드 대상)
    sim_scores = cosine_similarity(query_vector.reshape(1, -1), all_vectors_matrix)[0]
    
    # 3. 상위 인덱스 추출 (내림차순)
    top_indices = np.argsort(sim_scores)[::-1]
    
    # 4. 결과 리스트 생성 (필터링 적용)
    results = []
    for idx in top_indices:
        if len(results) >= k:
            # target_type K개를 모두 찾았으면 중단
            break
            
        item_name, item_type = all_nodes_list[idx]
        
        # (필터링 1) 자기 자신은 건너뛰기
        if item_name == query_name:
            continue
            
        # (필터링 2) 원하는 target_type이 아니면 건너뛰기
        if item_type != target_type:
            continue
            
        score = sim_scores[idx]
        results.append((item_name, item_type, score))
        
    return results

# --- [1번 작업: 키워드 10개 -> Top 20 '종목'] ---
print("\n" + "=" * 50)
print("🔍 [작업 1] 랜덤 키워드 10개에 대한 TOP 20 '종목' (Keyword-to-Stock)")
print("=" * 50)

N_QUERIES = 30

# 1. 쿼리할 키워드 10개 샘플링
if len(query_keywords) < N_QUERIES:
    sampled_keywords = query_keywords
else:
    sampled_keywords = np.random.choice(query_keywords, N_QUERIES, replace=False)

# 2. 10개 쿼리 실행
for i, query_name in enumerate(sampled_keywords):
    print(f"\n--- [1-{i+1:2d}] 쿼리: '{query_name}' (키워드) ---")
    
    start_time = time.time()
    # (수정) target_type='종목' 으로 설정
    top_k_results = find_top_k_filtered(query_name, '키워드', target_type='종목', k=10)
    
    if not top_k_results:
        print("  유사한 '종목' 노드를 찾지 못했습니다.")
        continue
        
    for rank, (name, n_type, score) in enumerate(top_k_results):
        print(f"  {rank+1:2d}위. {name} ({n_type}) (유사도: {score:.4f})")
    print(f"  (소요 시간: {time.time() - start_time:.2f}초)")


# --- [2번 작업: 종목 10개 -> Top 20 '키워드'] ---
print("\n" + "=" * 50)
print("📈 [작업 2] 랜덤 종목 10개에 대한 TOP 20 '키워드' (Stock-to-Keyword)")
print("=" * 50)

# 1. 쿼리할 종목 10개 샘플링
if len(query_stocks) < N_QUERIES:
    sampled_stocks = query_stocks
else:
    sampled_stocks = np.random.choice(query_stocks, N_QUERIES, replace=False)

# 2. 10개 쿼리 실행
for i, query_name in enumerate(sampled_stocks):
    print(f"\n--- [2-{i+1:2d}] 쿼리: '{query_name}' (종목) ---")
    
    start_time = time.time()
    # (수정) target_type='키워드' 로 설정
    top_k_results = find_top_k_filtered(query_name, '종목', target_type='키워드', k=10)
    
    if not top_k_results:
        print("  유사한 '키워드' 노드를 찾지 못했습니다.")
        continue
        
    for rank, (name, n_type, score) in enumerate(top_k_results):
        print(f"  {rank+1:2d}위. {name} ({n_type}) (유사도: {score:.4f})")
    print(f"  (소요 시간: {time.time() - start_time:.2f}초)")

print("\n" + "=" * 50)
print("모든 (필터링) 작업 완료.")


🔍 [작업 1] 랜덤 키워드 10개에 대한 TOP 20 '종목' (Keyword-to-Stock)

--- [1- 1] 쿼리: '권위' (키워드) ---
   1위. 현대건설 (종목) (유사도: 0.1273)
   2위. 드래곤플라이 (종목) (유사도: 0.1181)
   3위. 율촌 (종목) (유사도: 0.1119)
   4위. 뉴트리 (종목) (유사도: 0.1109)
   5위. 원익 (종목) (유사도: 0.1082)
   6위. 오공 (종목) (유사도: 0.0999)
   7위. 웹젠 (종목) (유사도: 0.0978)
   8위. 신풍 (종목) (유사도: 0.0937)
   9위. 선진 (종목) (유사도: 0.0897)
  10위. 웅진 (종목) (유사도: 0.0885)
  (소요 시간: 0.00초)

--- [1- 2] 쿼리: '비율' (키워드) ---
   1위. 쌍방울 (종목) (유사도: 0.2362)
   2위. 도부 (종목) (유사도: 0.2168)
   3위. KB금융 (종목) (유사도: 0.1989)
   4위. 사람인 (종목) (유사도: 0.1928)
   5위. 혜인 (종목) (유사도: 0.1923)
   6위. 디오 (종목) (유사도: 0.1905)
   7위. YTN (종목) (유사도: 0.1869)
   8위. 신원 (종목) (유사도: 0.1807)
   9위. SBS (종목) (유사도: 0.1789)
  10위. 엠브레인 (종목) (유사도: 0.1720)
  (소요 시간: 0.00초)

--- [1- 3] 쿼리: '타입' (키워드) ---
   1위. 엠브레인 (종목) (유사도: 0.2078)
   2위. 도부 (종목) (유사도: 0.1735)
   3위. 신원 (종목) (유사도: 0.1709)
   4위. 삼기 (종목) (유사도: 0.1641)
   5위. 씨앗 (종목) (유사도: 0.1487)
   6위. 진영 (종목) (유사도: 0.1476)
   7위. 대원 (종목) (유사도: 0.1328)
   8위. 쌍방울 (종목) (